In [10]:
import cv2
import numpy as np
from open3d import *

In [16]:
# intrinsics
fx = 718.856
fy = 718.856
cx = 607.1928
cy = 185.2157

# baseline
b = 0.573

left = cv2.imread('left.png', cv2.IMREAD_GRAYSCALE)
right = cv2.imread('right.png', cv2.IMREAD_GRAYSCALE)

sgbm = cv2.StereoSGBM_create(0, 96, 9, 8*9*9, 32*9*9, 1, 63, 10, 100, 32)
disparity = sgbm.compute(left, right)
disparity = disparity.astype(np.float32) / 16.0

pointcloud = []

rows, cols = left.shape

for v in range(left.shape[0]):
    for u in range(left.shape[1]):
        if not(disparity.item(v, u) <= 10.0 or disparity.item(v, u) >= 96.0):
            x = (u - cx) / fx
            y = (v - cy) / fy
            depth = fx * b / (disparity.item(v, u))
            point = np.array([x*depth, y*depth, depth, left.item(v, u)/255.0])      
            pointcloud.append(point);

In [17]:
cv2.imshow("left", left)
cv2.waitKey(0)

cv2.imshow('disparity', disparity / 96.0)
cv2.waitKey(0)
  
# closing all open windows
cv2.destroyAllWindows()

In [18]:
points = []
colors = []
for point in pointcloud:
    points.append(point[:3])
    colors.append([point[3]]*3)
pcd = geometry.PointCloud()
pcd.points = utility.Vector3dVector(points)
pcd.colors = utility.Vector3dVector(colors)
visualization.draw_geometries([pcd])